In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
import pickle

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [ ]:
from Ventilation_helper import *
import scipy.stats as stats
import sampling
from calculation import *

# Read the dataframes and the matching results 

In [ ]:
df_black = pd.read_csv("../venv/black_5_9.csv")
df_white = pd.read_csv("../venv/white_5_9.csv")

In [ ]:
# Read the counter-part index

name = "one_to_one_zero_100"
with open(name, "rb") as fp:   # Unpickling
    C_0_one_to_one = pickle.load(fp)
name = "one_to_one_one_100"
with open(name, "rb") as fp:   # Unpickling
    C_1_one_to_one = pickle.load(fp)


# Let's start splitting 

In [ ]:
df_black_one_to_one = df_black.iloc[C_0_one_to_one]
df_white_one_to_one = df_white.iloc[C_1_one_to_one]

In [ ]:
mask = ~df_black.index.isin(C_0_one_to_one)
df_black_left = df_black.loc[mask]

mask_white = ~df_white.index.isin(C_1_one_to_one)
df_white_left = df_white.loc[mask_white]

In [ ]:
# Spliting
# 1. Counter part, if minority is selected, then the majority match is also selected
# 2. For data not in the counterparts, just perform random spliting

In [ ]:
whole_lst = np.arange(len(df_black_one_to_one))

In [ ]:
df_counter_train_lst,df_counter_test_lst = split_with_match(df_black_one_to_one,df_white_one_to_one,5)

In [ ]:
df_black_left_train_lst, df_black_left_test_lst = split_without_match(df_black_left,5)
df_white_left_train_lst, df_white_left_test_lst = split_without_match(df_white_left,5)

# Random Forest as Demo

### User can choose their own models

In [ ]:
measure_columns = ['gender','rrt','gcs',
 'sofa_24hours',
 'heart_rate',
 'sbp_art',
 'dbp_art',
 'mbp_cuff',
 'resp_rate',
 'temperature',
 'spo2',
 'glucose',
 'admission_age',
 'charlson_comorbidity_index',
 'apsiii',
 'bmi']

In [ ]:
# Train a model for predicting ventilation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Create a Random Forest model

model_lst = []

for index in range(5):
    df_train = pd.DataFrame()
    df_train = pd.concat([df_train,df_counter_train_lst[index]])
    df_train = pd.concat([df_train,df_black_left_train_lst[index]])
    df_train = pd.concat([df_train,df_white_left_train_lst[index]])
    
    X_train = df_train[measure_columns]
    y_train = df_train['ventilation_status']
    
    # Use Smote sampling
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X_train, y_train)
    
    model = RandomForestClassifier(class_weight='balanced')

    # Train the model using the training data
    model.fit(X_res, y_res)
    model_lst.append(model)
    

In [ ]:
for i in range(len(model_lst)):
    y_pred_black_left = model_lst[i].predict(df_black_left_test_lst[i][measure_columns])
    y_pred_black_left_lst.append(y_pred_black_left)
    y_pred_white_left = model_lst[i].predict(df_white_left_test_lst[i][measure_columns])
    y_pred_white_left_lst.append(y_pred_white_left)
    y_pred_counterparts_black = model_lst[i].predict(df_counter_test_lst[i][df_counter_test_lst[i]['race_white_black']==1][measure_columns])
    y_pred_counterparts_black_lst.append(y_pred_counterparts_black)
    y_pred_counterparts_white = model_lst[i].predict(df_counter_test_lst[i][df_counter_test_lst[i]['race_white_black']==0][measure_columns])
    y_pred_counterparts_white_lst.append(y_pred_counterparts_white)

In [ ]:
"""
Conduct the fairness metric calculation based on diffrent groups/counterparts

""" 

y_pred_black_left_lst = []
y_pred_white_left_lst = []
y_pred_counterparts_black_lst = []
y_pred_counterparts_white_lst = []

y_true_black_left_lst = []
y_true_white_left_lst = []
y_true_counterparts_black_lst = []
y_true_counterparts_white_lst = []

res_total_lst = []
res_counter_part_lst = []
res_left_lst = []


for i in range(len(model_lst)):
    print(i)
    print("")
    y_pred_black_left = model_lst[i].predict(df_black_left_test_lst[i][measure_columns])
    y_pred_black_left_lst.append(y_pred_black_left)
    y_true_black_left_lst.append(df_black_left_test_lst[i]['ventilation_status'])
    
    
    y_pred_white_left = model_lst[i].predict(df_white_left_test_lst[i][measure_columns])
    y_pred_white_left_lst.append(y_pred_white_left)
    y_true_white_left_lst.append(df_white_left_test_lst[i]['ventilation_status'])
    
    
    y_pred_counterparts_black = model_lst[i].predict(df_counter_test_lst[i][df_counter_test_lst[i]['race_white_black']==1][measure_columns])
    y_pred_counterparts_black_lst.append(y_pred_counterparts_black)
    y_true_counterparts_black_lst.append(df_counter_test_lst[i][df_counter_test_lst[i]['race_white_black']==1]['ventilation_status'])
    
    
    y_pred_counterparts_white = model_lst[i].predict(df_counter_test_lst[i][df_counter_test_lst[i]['race_white_black']==0][measure_columns])
    y_pred_counterparts_white_lst.append(y_pred_counterparts_white)
    y_true_counterparts_white_lst.append(df_counter_test_lst[i][df_counter_test_lst[i]['race_white_black']==0]['ventilation_status'])
    
    total_black_pred_test = list(y_pred_counterparts_black_lst[i]) + list(y_pred_black_left_lst[i])
    total_white_pred_test = list(y_pred_counterparts_white_lst[i]) + list(y_pred_white_left_lst[i])
    total_pred = list(total_black_pred_test) + list(total_white_pred_test)
    total_race = np.array(list([1]*len(total_black_pred_test)) + list([0]*len(total_white_pred_test)))
    
    
    total_black_true_test = list(y_true_counterparts_black_lst[i]) + list(y_true_black_left_lst[i]) 
    total_white_true_test = list(y_true_counterparts_white_lst[i]) + list(y_true_white_left_lst[i]) 
    total_true = list(total_black_true_test) + list(total_white_true_test)
    
    
    y_black_counterpart_pred = list(y_pred_counterparts_black_lst[i])
    y_white_counterpart_pred = list(y_pred_counterparts_white_lst[i])
    
    y_black_counterpart_true = list(y_true_counterparts_black_lst[i])
    y_white_counterpart_true = list(y_true_counterparts_white_lst[i])
    
    counterpart_pred = list(y_black_counterpart_pred) + list(y_white_counterpart_pred)
    counterpart_true = list(y_black_counterpart_true) + list(y_white_counterpart_true)
    counterpart_race = np.array(list([1]*len(y_black_counterpart_pred)) + list([0]*len(y_white_counterpart_pred)))
    
    y_pred_black_left = list(y_pred_black_left_lst[i])
    y_pred_white_left = list(y_pred_white_left_lst[i])
    
    y_true_black_left = list(y_true_black_left_lst[i])
    y_true_white_left = list(y_true_white_left_lst[i])
    
    left_pred = list(y_pred_black_left) + list(y_pred_white_left)
    left_true = list(y_true_black_left) + list(y_true_white_left)
    left_race = np.array(list([1]*len(left_pred)) + list([0]*len(left_true)))
    
    
    
    res_total = calculate_fairness_metrics(np.array(total_true),np.array(total_pred),total_race)
    res_counter_part = calculate_fairness_metrics(np.array(counterpart_true),np.array(counterpart_pred),counterpart_race)
    
    res_total_lst.append(res_total)
    res_counter_part_lst.append(res_counter_part)
    
    

In [ ]:
"""
Calculate the DP gap based on the prediction probability of 
different attributes of Ventilation
""" 
l1_norm_total_lst = []
l1_norm_couner_lst = []
p_value_vent = []
y_pred_black_left_proba_lst = []
y_pred_white_left_proba_lst = []
y_pred_counterparts_black_proba_lst = []
y_pred_counterparts_white_proba_lst = []

for i in range(len(model_lst)):
    print(i)
    print("")
    y_pred_black_left_proba = model_lst[i].predict_proba(df_black_left_test_lst[i][measure_columns])
    y_pred_black_left_proba_lst.append(y_pred_black_left_proba)
    y_pred_white_left_proba = model_lst[i].predict_proba(df_white_left_test_lst[i][measure_columns])
    y_pred_white_left_proba_lst.append(y_pred_white_left_proba)

    y_pred_counterparts_black_proba = model_lst[i].predict_proba(df_counter_test_lst[i][df_counter_test_lst[i]['race_white_black']==1][measure_columns])
    y_pred_counterparts_black_proba_lst.append(y_pred_counterparts_black_proba)
    y_pred_counterparts_white_proba = model_lst[i].predict_proba(df_counter_test_lst[i][df_counter_test_lst[i]['race_white_black']==0][measure_columns])
    y_pred_counterparts_white_proba_lst.append(y_pred_counterparts_white_proba)

    y_pred_black_total_proba = np.vstack([y_pred_black_left_proba,y_pred_counterparts_black_proba])
    y_pred_white_total_proba = np.vstack([y_pred_white_left_proba,y_pred_counterparts_white_proba])

    y_pred_black_total_proba_mean = y_pred_black_total_proba.mean(axis=0)
    y_pred_white_total_proba_mean = y_pred_white_total_proba.mean(axis=0)

    y_pred_counterparts_black_proba_mean = y_pred_counterparts_black_proba.mean(axis=0)
    y_pred_counterparts_white_proba_mean = y_pred_counterparts_white_proba.mean(axis=0)


    l1_norm_total = np.linalg.norm(y_pred_black_total_proba_mean - y_pred_white_total_proba_mean, ord=1)
    l1_norm_couner = np.linalg.norm(y_pred_counterparts_black_proba_mean - y_pred_counterparts_white_proba_mean, ord=1)
    
    l1_norm_total_lst.append(l1_norm_total)
    l1_norm_couner_lst.append(l1_norm_couner)
    
    p_value_vent.append(vent_paired_t_test (y_pred_counterparts_black_proba,y_pred_counterparts_white_proba)[1])

In [ ]:
"""
Save Group based results into dictionary
"""
total_res = {}

for key in res_total_lst[0]:
    total_res[key] = []
    for i in range(len(res_total_lst)):
        total_res[key].append(res_total_lst[i][key])

In [ ]:
"""
Save counterpart based results into dictionary
"""
counterpart_res = {}

for key in res_counter_part_lst[0]:
    counterpart_res[key] = []
    for i in range(len(res_counter_part_lst)):
        counterpart_res[key].append(res_counter_part_lst[i][key])

In [ ]:
"""
Conduct the paired t-test on counterparts for ventilation prediction

"""

counter_part_vent_paired_t = vent_paired_t_test(y_pred_counterparts_black_proba,y_pred_counterparts_white_proba)